In [39]:
import json
import numpy as np
from transforms3d.quaternions import quat2mat
import copy

In [40]:
class MyJoint():
    def __init__(self,name):
        #main attribute
        self.name = name
        self.parent = self.parent_index = self.pos = None
        #not main 
        #self.children = []
        self.offset = self.channels = self.index = None
        self.matrix = None
        self.type = None

In [41]:
def read_line(stream, idx):
  if idx >= len(stream):
    return None, idx
  line = stream[idx]
  line= line[1:-2]
  line = line.replace('\t', '').replace(' ','')
  line = line.strip().split(',')
  idx += 1
  return line, idx

In [42]:
# [
# 	duration of frame in seconds (1D), 0
# 	root position (3D), 4
# 	root rotation (4D), 8
# 	chest rotation (4D), 12
# 	neck rotation (4D), 16
# 	right hip rotation (4D), 20
# 	right knee rotation (1D), 21
# 	right ankle rotation (4D), 25
# 	right shoulder rotation (4D), 29
# 	right elbow rotation (1D), 30
# 	left hip rotation (4D), 34
# 	left knee rotation (1D), 35
# 	left ankle rotation (4D), 39
# 	left shoulder rotation (4D), 43
# 	left elbow rotation (1D) 44
# ]

In [43]:
def get_motion(motion,line):
    motion["duration"] = line[0]
    motion["root_pos"] = line[1:4]
    motion["root"] = line[4:8]
    motion["chest"] = line[8:12]
    motion["neck"] = line[12:16]
    motion["right_hip"] = line[16:20]
    motion["right_knee"] = line[20:21]
    motion["right_ankle"] = line[21:25]
    motion["left_shoulder"] = line[25:29]
    motion["right_elbow"] = line[29:30]
    motion["right_hip"] = line[30:34]
    motion["right_knee"] = line[34:35]
    motion["right_ankle"] = line[35:39]
    motion["left_shoulder"] = line[39:43]
    motion["right_elbow"] = line[43:44]

In [44]:
def parse_motion_file(file_path):
    with open(file_path) as f:
        content = f.read().splitlines()
    content = content[4:]
    idx = 0
    eof = False
    motions = []
    motion = {}
    while(eof == False):
        line, idx = read_line(content, idx)
        if(line == None):
            eof = True
        elif(len(line) > 5):
            get_motion(motion,line)
            motions.append(copy.deepcopy(motion))
    return motions

In [45]:
def get_attach(data):
    attach = []
    attach.append(data["AttachX"])
    attach.append(data["AttachY"])
    attach.append(data["AttachZ"])
    return attach

In [46]:
file_path = "./data/deepmimic/humanoid3d.txt"
with open(file_path) as file:
    jdata = json.load(file)

In [47]:
joints = []
for data in jdata["Skeleton"]["Joints"]:
    joint = MyJoint(data["Name"])
    joint.index = data["ID"]
    joint.parent_index = data["Parent"]
    joint.type = data["Type"]
    joints.append(joint)

In [48]:
for data in jdata["BodyDefs"]:
    idx = data["ID"]
    joints[idx].offset = get_attach(data)
    print(joints[idx].name)

root
chest
neck
right_hip
right_knee
right_ankle
right_shoulder
right_elbow
right_wrist
left_hip
left_knee
left_ankle
left_shoulder
left_elbow
left_wrist


In [49]:
file_path = "./data/deepmimic/humanoid3d_backflip.txt"
motions = parse_motion_file(file_path)

In [50]:
for joint in joints:
    if(joint.parent_index != -1):
        joint.parent = joints[joint.parent_index]
    else:
        joint.parent = None

In [52]:
motion = motions[0]
for joint in joints:
    quat = motion[joint.name]
    quat = [float(x) for x in quat]
    quat = np.array(quat)
    print(quat,type(quat[0]))
    quat = np.deg2rad(quat)
    mat = quat2mat(quat)
    if(joint.parent == None):
        joint.pos = motion["root_pos"]
    else:
        joint.pos = 0

[ 9.99412e-01  2.92150e-02 -5.25000e-04 -1.79630e-02] <class 'numpy.float64'>
[9.99985e-01 4.32000e-04 5.72000e-04 5.50000e-03] <class 'numpy.float64'>
[ 0.9897717  0.00166   -0.011165  -0.140564 ] <class 'numpy.float64'>
[ 0.999678 -0.02092  -0.012925 -0.0063  ] <class 'numpy.float64'>
[-0.027859] <class 'numpy.float64'>


ValueError: not enough values to unpack (expected 4, got 1)

In [35]:
??quat2mat